In [93]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_local



In [100]:
image = cv2.imread('D:\\FCV\\Mini_project\\test.jpg')
orig = image.copy() 

In [ ]:
ch=input()

In [102]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
gray = cv2.GaussianBlur(gray, (5, 5), 0) 
edged = cv2.Canny(gray, 75, 200)
cv2.imshow("Gaus",gray)
cv2.imshow("canny",edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [103]:
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
print(cnts)


[array([[[565, 150]],

       [[564, 151]],

       [[560, 151]],

       ...,

       [[572, 155]],

       [[572, 151]],

       [[571, 150]]], dtype=int32), array([[[564, 151]],

       [[565, 150]],

       [[570, 150]],

       ...,

       [[554, 152]],

       [[559, 152]],

       [[560, 151]]], dtype=int32), array([[[460, 231]],

       [[459, 232]],

       [[454, 232]],

       ...,

       [[464, 235]],

       [[464, 232]],

       [[463, 231]]], dtype=int32), array([[[170, 270]],

       [[169, 271]],

       [[167, 271]],

       ...,

       [[175, 270]],

       [[174, 271]],

       [[171, 271]]], dtype=int32), array([[[671, 719]],

       [[670, 720]],

       [[666, 720]],

       [[665, 721]],

       [[661, 721]],

       [[660, 722]],

       [[656, 722]],

       [[655, 723]],

       [[651, 723]],

       [[650, 724]],

       [[646, 724]],

       [[645, 725]],

       [[640, 725]],

       [[639, 726]],

       [[635, 726]],

       [[634, 727]],

       [[63

In [104]:
for c in cnts:
    # approximating the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    if len(approx) == 4: # considering the case when the approximated contour has 4 points, which is the case when we have found the screen
        screenCnt = approx
        break

In [105]:
screenCnt

array([[[571, 150]],

       [[ 29, 229]],

       [[132, 986]],

       [[728, 860]]], dtype=int32)

cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) finds contours in the binary image edged. The cv2.RETR_LIST retrieval mode retrieves all of the contours and does not create any parent-child relationship between them.

cv2.CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments and leaves only their end points, which is sufficient for most cases.

contours now contains a list of contours found in the image.

sorted(contours, key=cv2.contourArea, reverse=True) sorts the list of contours based on their areas in descending order. The key parameter specifies that the sorting should be done based on the contour area, and reverse=True means sorting in descending order (from largest to smallest area).

[:5] selects the first five contours from the sorted list, which are the five largest contours in the image based on their areas.

In [106]:
print ("STEP 2: Finding the contours of the paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 2: Finding the contours of the paper


In [38]:
def order_points(pts):
    '''Rearrange coordinates to order:
      top-left, top-right, bottom-right, bottom-left'''
    rect = np.zeros((4, 2), dtype='float32')
    pts = np.array(pts)
    s = pts.sum(axis=1)
    # Top-left point will have the smallest sum.
    rect[0] = pts[np.argmin(s)]
    # Bottom-right point will have the largest sum.
    rect[2] = pts[np.argmax(s)]
 
    diff = np.diff(pts, axis=1)
    # Top-right point will have the smallest difference.
    rect[1] = pts[np.argmin(diff)]
    # Bottom-left will have the largest difference.
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect.astype('int').tolist()

In [77]:
def find_dest(pts):
    (tl, tr, br, bl) = pts
    # Finding the maximum width.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
 
    # Finding the maximum height.
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # Final destination co-ordinates.
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]
 
    return order_points(destination_corners)

In [107]:
screenCnt=sorted(np.concatenate(screenCnt).tolist())
screenCnt=order_points(screenCnt)
destination_corners=find_dest(screenCnt)


In [108]:
h, w = orig.shape[:2]
  # Getting the homography.
M = cv2.getPerspectiveTransform(np.float32(screenCnt), np.float32(destination_corners))
# Perspective transform using homography.
final = cv2.warpPerspective(orig, M, (destination_corners[2][0], destination_corners[2][1]),
                                flags=cv2.INTER_LINEAR)

In [109]:
cv2.imshow("Final image",final)
cv2.waitKey(0)
cv2.destroyAllWindows()